In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [5]:
def sequential_generator(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=2:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [6]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [7]:
n_labels =2

In [13]:
model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
model = Model(model_input_embedding, model_output)
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 75, 200)      320800      input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_6 (LeakyReLU)       (None, 75, 200)      0           bidirectional_2[0][0]            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 73, 200)      120200      leaky_re_lu_6[0][0]              
__________________________________________________________________________________________________
conv1d_5 (

In [14]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt')]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt')]

In [15]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [16]:
def sequential_generator_germeval(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=3:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [12]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [ ]:
model.reset_states()
model.fit_generator(
    sequential_generator_germeval('/home/gwiedemann/notebooks/OffLang/sample_train.txt', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = sequential_generator_germeval('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', batch_size),
    validation_steps = math.ceil(len(dev_sentences) / batch_size),
    callbacks = [checkpoint], 
)

Epoch 1/50
132/132 [==============================] - 33s 249ms/step - loss: 0.0355 - acc: 0.9870 - val_loss: 1.2422 - val_acc: 0.8029

Epoch 00001: val_acc improved from 0.79688 to 0.80288, saving model to best_classification_model.h5
Epoch 2/50
132/132 [==============================] - 34s 254ms/step - loss: 0.0326 - acc: 0.9875 - val_loss: 1.3217 - val_acc: 0.7861

Epoch 00002: val_acc did not improve from 0.80288
Epoch 3/50
132/132 [==============================] - 34s 256ms/step - loss: 0.0284 - acc: 0.9905 - val_loss: 1.5153 - val_acc: 0.7897

Epoch 00003: val_acc did not improve from 0.80288
Epoch 4/50
132/132 [==============================] - 33s 249ms/step - loss: 0.0370 - acc: 0.9867 - val_loss: 1.2796 - val_acc: 0.7981

Epoch 00004: val_acc did not improve from 0.80288
Epoch 5/50
132/132 [==============================] - 34s 257ms/step - loss: 0.0292 - acc: 0.9889 - val_loss: 1.5190 - val_acc: 0.7800

Epoch 00005: val_acc did not improve from 0.80288
Epoch 6/50
132/132 [

In [18]:
testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
for i in range(len(dev_sentences)):
    testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
results = model.predict(testset_features)

In [19]:
idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
predLabels = results.argmax(axis=-1)
devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
# print(idx2Label)
# print(predLabels)
# print(devLabels)
f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
a = accuracy_score(devLabels, predLabels)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))

Test-Data: Prec: 0.678, Rec: 0.727, F1: 0.702, Acc: 0.796
